## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: string (nullable = true)

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from members limit 3

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,GUEST,GUEST,GUEST,0,(000) 000-0000,null,2012-07-01 00:00:00
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,null,2012-07-02 12:02:05
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,null,2012-07-02 12:08:23


In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select name from facilities
where membercost <> 0

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
select count(name) from facilities
where membercost = 0

count(name)
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
select facid, name as facilityname, membercost, monthlymaintenance from facilities
where membercost <> 0 and membercost <= monthlymaintenance * 0.2

facid,facilityname,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
select * from facilities
where facid in (1,5)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT name as facility_name, monthlymaintenance,
CASE
    WHEN monthlymaintenance <= 100 THEN 'Cheap'
    ELSE 'Expensive'
END AS cost_label
FROM facilities;

facility_name,monthlymaintenance,cost_label
Tennis Court 1,200,Expensive
Tennis Court 2,200,Expensive
Badminton Court,50,Cheap
Table Tennis,10,Cheap
Massage Room 1,3000,Expensive
Massage Room 2,3000,Expensive
Squash Court,80,Cheap
Snooker Table,15,Cheap
Pool Table,15,Cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
SELECT *, CAST(joindate AS TIMESTAMP) as joindate2 from members 
WHERE NOT firstname = 'GUEST'
order by joindate2 desc

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate,joindate2
37,Smith,Darren,"3 Funktown, Denzington, Boston",66796,(822) 577-3541,null,2012-09-26 18:08:45,2012-09-26T18:08:45.000+0000
36,Crumpet,Erica,"Crimson Road, North Reading",75655,(811) 732-4816,2,2012-09-22 08:36:38,2012-09-22T08:36:38.000+0000
35,Hunt,John,"5 Bullington Lane, Boston",54333,(899) 720-6978,30,2012-09-19 11:32:45,2012-09-19T11:32:45.000+0000
33,Tupperware,Hyacinth,"33 Cheerful Plaza, Drake Road, Westford",68666,(822) 665-5327,null,2012-09-18 19:32:05,2012-09-18T19:32:05.000+0000
30,Purview,Millicent,"641 Drudgery Close, Burnington, Boston",34232,(855) 941-9786,2,2012-09-18 19:04:01,2012-09-18T19:04:01.000+0000
29,Worthington-Smyth,Henry,"55 Jagbi Way, North Reading",97676,(855) 894-3758,2,2012-09-17 12:27:15,2012-09-17T12:27:15.000+0000
28,Farrell,David,"437 Granite Farm Road, Westford",43532,(855) 755-9876,null,2012-09-15 08:22:05,2012-09-15T08:22:05.000+0000
27,Rumney,Henrietta,"3 Burkington Plaza, Boston",78533,(822) 989-8876,20,2012-09-05 08:42:35,2012-09-05T08:42:35.000+0000
26,Jones,Douglas,"976 Gnats Close, Reading",11986,844 536-8036,11,2012-09-02 18:43:05,2012-09-02T18:43:05.000+0000
24,Sarwin,Ramnaresh,"12 Bullington Lane, Boston",65464,(822) 413-1470,15,2012-09-01 08:44:42,2012-09-01T08:44:42.000+0000


In [0]:
%sql
SELECT name, joindate2 FROM (
SELECT name, ROW_NUMBER() OVER (ORDER BY joindate2 DESC) AS rownumber, joindate2 
from 
(SELECT CONCAT(firstname,' ',surname) as name, CAST(joindate AS TIMESTAMP) as joindate2 from members 
WHERE NOT firstname = 'GUEST') ) as foo
where rownumber = 1

name,joindate2
Darren Smith,2012-09-26T18:08:45.000+0000


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
select CONCAT(tt.name,' ',tt.membername) as fac_member from
(select distinct t.name, t.membername from 
(select c.bookid, c.facid, c.name, c.memid, CONCAT(m.firstname,' ',m.surname) as membername
from (select b.bookid, b.facid, f.name, b.memid from bookings as b
left join facilities as f on f.facid = b.facid) as c
left join members as m on m.memid = c.memid) as t 
where t.name LIKE '%Tennis Court%' and t.membername NOT LIKE '%GUEST%'
order by t.membername) as tt

fac_member
Tennis Court 2 Anne Baker
Tennis Court 1 Anne Baker
Tennis Court 2 Burton Tracy
Tennis Court 1 Burton Tracy
Tennis Court 1 Charles Owen
Tennis Court 2 Charles Owen
Tennis Court 2 Darren Smith
Tennis Court 2 David Farrell
Tennis Court 1 David Farrell
Tennis Court 2 David Jones


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
select b.bookid, CONCAT(f.name,' ',m.firstname,' ',m.surname) as fac_member,
CASE
    WHEN b.memid = 0 THEN f.guestcost * b.slots
    ELSE f.membercost * b.slots
END AS daily_cost
from bookings as b
left join facilities as f on b.facid = f.facid
left join members as m on b.memid = m.memid
where CAST(b.starttime AS Date) = '2012-09-14' and 
CASE
    WHEN b.memid = 0 THEN f.guestcost * b.slots
    ELSE f.membercost * b.slots
END > 30
order by daily_cost DESC

bookid,fac_member,daily_cost
2946,Massage Room 2 GUEST GUEST,320.0
2937,Massage Room 1 GUEST GUEST,160.0
2942,Massage Room 1 GUEST GUEST,160.0
2940,Massage Room 1 GUEST GUEST,160.0
2926,Tennis Court 2 GUEST GUEST,150.0
2925,Tennis Court 2 GUEST GUEST,75.0
2922,Tennis Court 1 GUEST GUEST,75.0
2920,Tennis Court 1 GUEST GUEST,75.0
2948,Squash Court GUEST GUEST,70.0
2941,Massage Room 1 Jemima Farrell,39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
select bb.bookid, CONCAT(bb.name,' ',m.firstname,' ',m.surname) as fac_member, bb.daily_cost from
(SELECT *,
CASE
    WHEN b.memid = 0 THEN f.guestcost * b.slots
    ELSE f.membercost * b.slots
END AS daily_cost
from bookings as b
left join facilities as f on b.facid = f.facid
where CAST(b.starttime AS Date) = '2012-09-14') as bb
left join members as m on bb.memid = m.memid
where bb.daily_cost > 30
order by bb.daily_cost DESC

bookid,fac_member,daily_cost
2946,Massage Room 2 GUEST GUEST,320.0
2937,Massage Room 1 GUEST GUEST,160.0
2942,Massage Room 1 GUEST GUEST,160.0
2940,Massage Room 1 GUEST GUEST,160.0
2926,Tennis Court 2 GUEST GUEST,150.0
2925,Tennis Court 2 GUEST GUEST,75.0
2922,Tennis Court 1 GUEST GUEST,75.0
2920,Tennis Court 1 GUEST GUEST,75.0
2948,Squash Court GUEST GUEST,70.0
2941,Massage Room 1 Jemima Farrell,39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
select * from (select f.name, sum(
CASE
    WHEN b.memid = 0 THEN f.guestcost * b.slots
    ELSE f.membercost * b.slots
END) AS total_revenue
from bookings as b
left join facilities as f on b.facid = f.facid
left join members as m on b.memid = m.memid
group by f.name
order by total_revenue desc) as t
where total_revenue < 1000

name,total_revenue
Pool Table,270.0
Snooker Table,240.0
Table Tennis,180.0
